### Introduction:

The IceCube Neutrino Observatory is composed of many light detectors, called "DOMs" (digital optical modules), buried two kilometers deep below the surface of the South Pole. These light detectors are arranged in a roughly three dimensional grid: there are many "strings" hanging vertically downwards in the ice, and on each string there are many DOMs: 

![](../resources/images/I3det_v2_edited.jpeg)

When a high energy charged particle passes through the ice, it produces what is called Cherenkov light, which can then be detected by the many DOMs. The **locations** of the DOMs that see light, the **times** at which they saw light, and the **amount** of light they see all communicate information about this charged particle.

As experimentalists, what we want to do is work backwards: if what we have is this information about the light seen by the detector, what can we figure out about the original particle? 

### 1: Visualizing events:

For this activity, we've prepared a simulation of many events within the IceCube detector. Execute the cell below by clicking the triangle or hitting `^ enter` to download the simulation from google drive.

In [ ]:
# download tracks.parquet
!gdown 1nqffw6xHLdX2oO8d-5xjExYZryo7rp3x

# download pre-prepared analysis code
!git clone "https://github.com/kcarloni/IceCube_MasterClass_at_Harvard2024";

import sys
sys.path.insert(0, "./IceCube_MasterClass_at_Harvard2024/src/")

In [ ]:
from event_reader import EventSelection, Event
tracks = EventSelection("tracks.parquet")

Let's look at an 'event' in the detector. An event is just a bunch of 'hits', ie. instances in which light hit different sensors, which are grouped together because they all occured within the same time window.

In [ ]:
evt_num = 24
evt = tracks[evt_num]
evt

We can visualize an event within the detector by plotting it.  

In [ ]:
import plotly.graph_objs as go
from plot_event import *

layout = get_3d_layout()
plot_det = get_I3det()

fig = go.FigureWidget(data=plot_det, layout=layout)

plot_evt = plot_first_hits(evt)
fig.add_trace(plot_evt)

fig

### 2: From what direction did the neutrino come from? 

WIP: Felix's directional reconstruction game. 

In [ ]:
!pip install "analysis @ git+https://github.com/jlazar17/IceCube_Masterclass_MoonShadow";

from analysis import DataReader
from analysis import reco_game

tracks = DataReader("tracks.parquet")

In [ ]:
reco_game(tracks)

### 3: 

WIP: implement an optimizer-based directional reco?

WIP (?) implement / load some kind of ML reco to show? 

WIP: make plots to show how different reconstructions compare; performance as a function of energy...